## Notebook Description

This is "experimental" work to get the data transformation process down separate from the scraping logic.

In [4]:
import numpy as np
import pandas as pd
from datetime import datetime

Constants

In [5]:
dates = ['May 10, 2021', 'May 11, 2021'] # some fake dates

# Columns
pool_participant_id = 'Pool Participant ID'
asset_type = 'Asset Type'
asset_id = 'Asset ID'
date = 'Date'
last_updated = 'Last Updated'
hour_closing_price = 'Hour Closing Price' 
hour = 'Hour'
energy_unit = 'MWh'
peak_status = 'Peak Status'
hours_row = ['Hour 1', 'Hour 2', 'Hour 3', 'Hour 4', 'Hour 5', 'Hour 6', 'Hour 7', 'Hour 8', 'Hour 9', 'Hour 10', 'Hour 11', 'Hour 12', 'Hour 13', 'Hour 14', 'Hour 15', 'Hour 16', 'Hour 17', 'Hour 18', 'Hour 19', 'Hour 20', 'Hour 21', 'Hour 22', 'Hour 23', 'Hour 24']

# Choose columns
original_data_index = [pool_participant_id, asset_type, asset_id]
columns_in_scrape = [pool_participant_id, asset_type, asset_id] + hours_row
drop_columns = [pool_participant_id, asset_type, hour_closing_price]
select_columns = [asset_id, date, hour, energy_unit, peak_status, last_updated]
sort_transform_by = [asset_id, date, hour]

# Filters
desired_sites = ['941A', '941C']

'''
Peak hours: 7 am - 11 am
Hour Endings:
    Hr 1: 12:00 am - 12:59 am
    Hr 2: 1:00 am -1:59 am
    Hr 3: 2:00 am - 2:59 am
    ...
    Hr 7: 6:00 am - 6:59 am (Off-Peak)
    Hr 8: 7:00 am - 7:59 am (Peak)
    ...
    Hr 10: 9:00 - 9:59 am (Peak)
    Hr 11: 10:00 am - 10:59 am (Peak)
    Hr 12: 11:00 am - 11:59 am (Off-Peak)
'''
peak_hours_start = 8 # 7:00 am
peak_hours_end = 11 # 10:59 am

Generate fake data

In [6]:
hour_values = ['45.45', '48.64', '47.34', '44.89', '43.44', '70.49', '218.73', '438.24', '558.40', '552.43', '580.21', '506.76', '377.32', '168.27', '289.80', '325.83', '499.20', '627.39', '567.60', '484.54', '94.33', '65.78', '46.82', '44.73']
for i in range(len(hour_values)):
    hour_values[i] = float(hour_values[i])

fake_data = np.random.rand(24).tolist()

all_data = [[['9496', 'RETAILER', '941A'], 
             ['9496', 'RETAILER', '941B']],
            [['9496', 'RETAILER', '941C'], 
             ['9496', 'RETAILER', '941D']]]

for d_idx, day_of_data in enumerate(all_data):
    for r_idx, row in enumerate(day_of_data):
        all_data[d_idx][r_idx].extend(fake_data)

Transform all data

In [10]:
def _load_data_from_list(day_data, hour_values):
    '''
    Loads data from list into Pandas.
    
    
    Parameters:
        day_data:
        hour_values:
        
        
    Returns:
        A Pandas DataFrame version of the AESO Report.
    
    '''
    
    # Load data into Pandas
    original_data = pd.DataFrame(data=day_data, columns=columns_in_scrape)
    original_data = original_data.set_index(original_data_index)
    
    # the result in the next line is exactly the table from the AESO report with the multi-level header
    original_data.columns = pd.MultiIndex.from_tuples(zip(hours_row, hour_values))
    
    return original_data


def _filter_data(data, filters):
    '''
    Filter data.
    

    Parameters:
        data:
        filters:
    
    
    Returns:
    '''
    # Picks any from col 1 & 2 of the index, the chosen sites, and all of the non-index columns
    filtered_data = data.loc[(slice(None), slice(None), filters['sites']), :]
    
    return filtered_data
    
    
def _transform(data, processing_date):
    '''
    Transforms data into a row wise format according to specifications.
    
    Parameters:
    
    Returns:
        A transformed Pandas DataFrame.
    '''
    
    # This moves the closing prices and hours into columns
    # Reset index from original_data_index so that each index value appears in each row
    stacked_result = data.stack([0, 1]).reset_index() 
    
    # Rename columns
    stacked_result.columns = [pool_participant_id, asset_type, asset_id, hour, hour_closing_price, energy_unit]

    # Extract integer from Hour column and add Peak Status and Date columns
    stacked_result[hour] = stacked_result.apply(lambda row: row[hour].split(' ')[1], axis=1).astype(int) 
    stacked_result[peak_status] = np.where((stacked_result[hour] >= peak_hours_start) & 
                                             (stacked_result[hour] <= peak_hours_end), 'Peak', 'Off-Peak')
    stacked_result[date] = processing_date
    stacked_result[last_updated] = datetime.today().strftime('%Y-%m-%d')

    # Select desired columns in given order then sort the rows
    target = stacked_result[select_columns]
    target = target.sort_values(sort_transform_by).reset_index(drop=True)
    
    return target
    
    
def _process_date(day_data, process_date, hour_values):
    '''
    Performs tranformations for a single day.
    
    
    Parameters:
        day_data: The raw data for the current `process_date`.
        process_date: The current date that is being processed.
        hour_values: The closing prices for each hour
        
        
    Returns: 
        A transformed dataframe for the date that was processed with schema defined by `select_columns`:
            ['Asset ID', 'Date', 'Hour', 'MWh', 'Peak Status', 'Last Updated']
    '''
    
    loaded_data = _load_data_from_list(day_data, hour_values)
    
    filtered_data = _filter_data(loaded_data, filters={
        'sites': desired_sites
    })
    
    transformed_data = _transform(filtered_data, process_date)
    
    return transformed_data

def process_all_dates(all_data, dates):
    '''
    Takes in a list of data and dates (parsed from the AESO website) and
        returns a transformed Pandas DataFrame.


    Parameters:
        all_data: 1 list per date, each of which consists of a list of rows.
        dates: list of dates


    Returns:
        A transformed Pandas DataFrame for all dates with schema defined by `select_columns`:
            ['Asset ID', 'Date', 'Hour', 'MWh', 'Peak Status', 'Last Updated']
    '''
    
    resultant_data = pd.DataFrame(columns=select_columns)
    
    for i, processing_date in enumerate(dates):       
        # Convert from column to row wise data
        target = _process_date(all_data[i], processing_date, hour_values)

        # Append the current processing_date's data to the result
        resultant_data = pd.concat([resultant_data, target], axis=0).reset_index(drop=True)
        
    return resultant_data

In [11]:
result = process_all_dates(all_data, dates)

In [12]:
pd.set_option('display.max_rows', len(result))
result

,Asset ID,Date,Hour,MWh,Peak Status,Last Updated
0,941A,"May 10, 2021",1,0.186252,Off-Peak,2021-06-07
1,941A,"May 10, 2021",2,0.712487,Off-Peak,2021-06-07
2,941A,"May 10, 2021",3,0.592763,Off-Peak,2021-06-07
3,941A,"May 10, 2021",4,0.531404,Off-Peak,2021-06-07
4,941A,"May 10, 2021",5,0.356793,Off-Peak,2021-06-07
5,941A,"May 10, 2021",6,0.748715,Off-Peak,2021-06-07
6,941A,"May 10, 2021",7,0.695102,Off-Peak,2021-06-07
7,941A,"May 10, 2021",8,0.188544,Peak,2021-06-07
8,941A,"May 10, 2021",9,0.990849,Peak,2021-06-07
9,941A,"May 10, 2021",10,0.608359,Peak,2021-06-07


In [13]:
result.dtypes

Asset ID         object
Date             object
Hour             object
MWh             float64
Peak Status      object
Last Updated     object
dtype: object